In [104]:
import lightgbm
import pandas as pd
import numpy as np

In [ ]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


In [ ]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

# Try reduce columns

In [ ]:
#selected = pd.concat([init_record.filter(regex=r'allrows', axis=1), init_record["response_variable"]], axis = 1)
#selected = init_record.drop(init_record.filter(regex=r'allrows', axis=1),axis = 1)
#selected

In [ ]:
def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable']
    x = data.drop('response_variable',axis='columns')
    return x, y

feature, label = construct_feature_label(init_record)


# PCA

In [ ]:
from sklearn import decomposition


In [ ]:
#pca = decomposition.PCA(n_components=0.9)
#feature_pca = pca.fit_transform(feature)

# Model

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,accuracy_score


In [ ]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(feature)

In [ ]:
y_hat = model.predict(feature)

In [100]:
y_hat = np.where(y_hat == -1, 1, 0) 

In [101]:
f1_score(label, y_hat)

0.33315948680504853

In [102]:
confusion_matrix(label,y_hat)

array([[93926,  7797],
       [ 4989,  3194]])

In [103]:
accuracy_score(label,y_hat)

0.8836642221534766

In [68]:
# Try rollout Data